In [7]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [8]:
import psycopg2
con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cmpdmnkfrh2m.us-east-2.redshift.amazonaws.com', 
port= '5439', user= 'mantix', password= 'Mantix!2')

# import psycopg2
# con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cmpdmnkfrh2m.us-east-2.redshift.amazonaws.com', 
# port= '5439', user= 'mantix', password= 'Mantix!2')

# cur = con.cursor()
# cur.execute("select * from cdp.orders_data;")
# orders_data = pd.read_sql("select * from cdp.orders_data;",con)

In [9]:
rules_data = pd.read_sql("select * from cdp.often_bundled_rules;",con)
rules_data.shape


,rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id
0,1,859845,859842,3M REPLACEMENT FILTER FAPF-F2N-4,FILTRETE ROOM AIR PURIFIER - LARGE ROOM - FAP-...,0.000182,0.000391,0.000104,0.571429,1461.371429,0.000104,2.332420948359011,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
1,3,797444,211203,5 GAL. HEAVY DUTY STORAGE BIN BLACK/YELLOW,MEMBERSHIP: $5.00,0.000391,0.087798,0.000130,0.333333,3.796615,0.000096,1.3683037459920229,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
2,5,665868,552855,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,FESTIVAL PETITE DILL PICKLES 46 OZ,0.000130,0.000652,0.000104,0.800000,1227.552000,0.000104,4.996741482234563,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
3,7,665868,552526,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,"MOTTS, REALEMON, 100% LEMON JUICE, 2.5 OZ",0.000130,0.000104,0.000104,0.800000,7672.200000,0.000104,4.999478637157531,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
4,9,665868,818591,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,SMUCKERS CONCORD GRAPE JELLY- 18 OZ,0.000130,0.000182,0.000104,0.800000,4384.114286,0.000104,4.999087615025678,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123


In [10]:
rules_data.head(100)

,rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id
0,1,859845,859842,3M REPLACEMENT FILTER FAPF-F2N-4,FILTRETE ROOM AIR PURIFIER - LARGE ROOM - FAP-...,0.000182,0.000391,0.000104,0.571429,1461.371429,0.000104,2.332420948359011,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
1,3,797444,211203,5 GAL. HEAVY DUTY STORAGE BIN BLACK/YELLOW,MEMBERSHIP: $5.00,0.000391,0.087798,0.000130,0.333333,3.796615,0.000096,1.3683037459920229,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
2,5,665868,552855,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,FESTIVAL PETITE DILL PICKLES 46 OZ,0.000130,0.000652,0.000104,0.800000,1227.552000,0.000104,4.996741482234563,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
3,7,665868,552526,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,"MOTTS, REALEMON, 100% LEMON JUICE, 2.5 OZ",0.000130,0.000104,0.000104,0.800000,7672.200000,0.000104,4.999478637157531,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
4,9,665868,818591,A TASTE OF THAI COCONUT MILK 13.5 FL OZ,SMUCKERS CONCORD GRAPE JELLY- 18 OZ,0.000130,0.000182,0.000104,0.800000,4384.114286,0.000104,4.999087615025678,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,150,618569,615187,BOBS RED MILL UNBLEACHED WHITE FLOUR 5 LB,"KRAFT STOVE TOP CHICKEN STUFFING MIX, 2 CT - 1...",0.001069,0.001095,0.000104,0.097561,89.108014,0.000103,1.1068948826827922,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,None,TNB00084,STOM000000123
96,152,618569,439327,BOBS RED MILL UNBLEACHED WHITE FLOUR 5 LB,LINDSAY LARGE CALIFORNIA RIPE PITTED OLIVES - ...,0.001069,0.000704,0.000104,0.097561,138.612466,0.000104,1.1073281774775479,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
97,154,618569,428134,BOBS RED MILL UNBLEACHED WHITE FLOUR 5 LB,"MD BATH TISSUE, 12 DOUBLE ROLL",0.001069,0.002268,0.000182,0.170732,75.280908,0.000180,1.2031474981483952,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
98,156,764107,428134,"BRAWNY, PAPER TOWELS, 6 LARGE ROLL","MD BATH TISSUE, 12 DOUBLE ROLL",0.000417,0.002268,0.000130,0.312500,137.790948,0.000129,1.4512466496512795,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123


In [6]:
rules_data.shape

(629, 16)

In [3]:
cur = con.cursor()
#cur.execute("select * from cdp.orders_data;")
#orders_data = pd.read_sql("select * from cdp.orders_data;",con)

In [4]:
orders_data = pd.read_csv('orders_data.csv')
orders_data.shape

(470571, 34)

In [5]:
orders_data['order_number']=orders_data['order_number'].astype(int).astype(str)
orders_data.head()

,order_number,amount_refunded,amount_available_for_refund,item_tax_total,shipping_tax_total,tax_total,discount_total,shipping_subtotal,shipping_total,subtotal,...,billing_address_country_code,billing_address_type,fulfillment_shipping_method_code,fulfillment_shipping_method_name,kibo_tenant_id,kibo_site_id,tenant_id,store_id,submitted_date,fulfillment_date
0,336121,0.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,Cape Verde,Residential,60255cdb46f0abccb6687ad9,Flat Rate,29107,47751,TNG00072,STIN000000110,2019-06-07,2019-04-18
1,338138,0.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,Luxembourg,Residential,60255cdcf040ba8cd83d8960,Flat Rate,29107,47751,TNG00072,STIN000000110,2020-06-04,2019-03-14
2,338638,0.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,Poland,Residential,60255cdc46ab13f632f70f1a,Flat Rate,29107,47751,TNG00072,STIN000000110,2020-12-16,2016-04-11
3,338027,0.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,Algeria,Residential,60255cdc4afd39129788a766,Flat Rate,29107,47751,TNG00072,STIN000000110,2020-01-04,2019-08-18
4,338559,0.0,NaN,NaN,NaN,NaN,NaN,0,0,NaN,...,Wallis and Futuna Islands,Residential,60255cdc59e7faf4dabd0437,Flat Rate,29107,47751,TNG00072,STIN000000110,2019-10-24,2020-09-05


In [6]:
#orders_data.dtypes

In [7]:
products_data = pd.read_sql("select * from cdp.products_data;",con)

In [8]:
products_data.columns

Index(['product_code', 'product_name', 'product_usage', 'brand',
       'product_short_description', 'product_description', 'upc',
       'manufacturer', 'list_price', 'sale_price', 'currency_code',
       'supplier_cost', 'supplier_currency_code', 'supplier_mfg_part_number',
       'is_variation', 'product_type_id', 'linked_category_ids', 'tenant_id',
       'store_id', 'update_date', 'create_date', 'category_id', 'image_url',
       'product_image_id'],
      dtype='object')

In [9]:
prod_url = products_data[['product_name','image_url']].drop_duplicates()
prod_url=dict(zip(prod_url.product_name,prod_url.image_url))

In [10]:
items_data = pd.read_sql("select * from cdp.items_data;",con)


In [11]:
items_data['order_number']=items_data['order_number'].astype(int).astype(str)
items_data.shape

(54480, 21)

In [12]:
items_data.head()

,order_number,list_price,sale_price,extended_price,discount_total,total,product_code,product_name,quantity,discount_id,...,fulfillment_location_code,fulfillment_status,fulfillment_method,tenant_id,store_id,update_date,create_date,variant_product_code,product_usage,purchase_location
0,399,4.79,3.79,3.79,0.0,3.79,418021,DENTAL FLOSS,1,NaN,...,613,Pending,Pickup,TNT00076,STOM000000114,None,None,None,Standard,613
1,221,2.99,NaN,2.99,0.0,2.99,856800,EZ REACH LIGHTER,1,NaN,...,601,Pending,Pickup,TNT00076,STOM000000114,2021-02-05,2021-02-05,None,Standard,601
2,897,22.99,NaN,22.99,0.0,22.99,9520005,MENS STRT LEG REG JEAN,1,NaN,...,613,Pending,Pickup,TNT00076,STOM000000114,2021-05-28,2021-05-28,9541713,Configurable,613
3,271,44.97,NaN,44.97,0.0,44.97,752881,LI ION DRILL DRIVER,1,NaN,...,613,PendingFulfillment,Pickup,TNT00076,STOM000000114,2021-02-11,2021-02-11,None,Standard,613
4,327,4.79,3.79,3.79,0.0,3.88,418021,DENTAL FLOSS,1,NaN,...,613,PendingFulfillment,Pickup,TNT00076,STOM000000114,2021-02-23,2021-02-23,None,Standard,613


In [13]:
items_data.columns

Index(['order_number', 'list_price', 'sale_price', 'extended_price',
       'discount_total', 'total', 'product_code', 'product_name', 'quantity',
       'discount_id', 'discount_name', 'fulfillment_location_code',
       'fulfillment_status', 'fulfillment_method', 'tenant_id', 'store_id',
       'update_date', 'create_date', 'variant_product_code', 'product_usage',
       'purchase_location'],
      dtype='object')

In [14]:
orders_data_syn=  orders_data[(orders_data['tenant_id']=='TNB00084')]

In [15]:
orders_data_syn.shape

(47805, 34)

In [16]:
#orders_data_syn.order_number = orders_data_syn.order_number.astype(str)
#items_data.order_number = items_data.order_number.astype(str)

In [17]:
pro =pd.merge(orders_data_syn,items_data[['product_name','product_code','sale_price','total','order_number','quantity']],on=['order_number'],how='left')

In [18]:
final_pro = pro[['order_number','product_name']]

final_pro = final_pro.dropna()


In [19]:
basket = final_pro.groupby('order_number', as_index=False).agg({'product_name': lambda x: list(x)})
basket = basket.set_index('order_number')

In [15]:
basket.shape

(38361, 1)

In [16]:
#basket['prod_name_str'] = basket['product_name'].astype(str)
#basket = basket[basket.prod_name_str!='[nan]'][['product_name']]

In [17]:
basket.head()

,product_name
order_number,
1000,[COMMERCIAL CHEF .9 MICROWAVE]
1001,[COMMERCIAL CHEF .9 MICROWAVE]
1002,"[LOLA STICKY MOP REFILL, TERRO T300 ANT KILLER..."
1003,"[OUTDOOR PORTABLE GAS FIRE PIT, MEMBERSHIP: $5..."
1004,"[AUDUBON PARK WILD BIRD FOOD 10 LBS, PHILLIPS ..."


In [18]:
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer()
basket = basket.join(pd.DataFrame(mlb.fit_transform(basket.pop('product_name')),
                                  columns=mlb.classes_,
                                  index=basket.index))
basket.head()

,#10-24 INSERT NUT #880548,#10-24 X 2 ZINC THUMB SCREW #880947,#6 X 5/8 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882624,#8 X 3/4 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882625,1 AMP USB WALL CHARGER BLACK,1 INCH BRASS PLATED NON-REMOVABLE PIN HINGES 2 COUNT,1 PACK FLUIDMASTER 400CRP14 TOILET FILL VALVE AND FLAPPER REPAIR KIT,1-1/16 INCH THREADED FURNITURE LEVELING GLIDES 4 COUNT,1-1/2 INCH BROWN ROUND SELF-STICK FELT PADS 24 COUNT,1-1/8 INCH OATMEAL NAIL-ON FURNITURE GLIDES WITH FELT 8 COUNT,...,ZIPLOC TWIST N LOCK STORAGE CONTAINER 16 OZ 3 COUNT,ZONE PERFECT KETO BARS CHOCOLATE HAZELNUT 5 PK,ZOOM 001-004 U-TALE WORM PLUM,ZOOM 011045 FAT ALBERT GRUB WHITE PEARL,"ZOTZ FIZZ POWER CANDY, .7OZ STRIP CANDY (FLAVORS VARY)","ZOTZ FIZZ, POWER CANDY STRINGS, 0.7 OUNCE (FLAVORS VARY)",ZYRTEC ALLERGY RELIEF 24 CT,ZYRTEC ANTIHISTAMINE 10 MG LIQUID GELS 25 CT,membership_product,roadpower PREMIUM BATTERY TESTER
order_number,,,,,,,,,,,,,,,,,,,,,
1000,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1001,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1002,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1003,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1004,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
basket.shape

(38361, 8494)

In [20]:
frequent_itemsets = apriori(basket, min_support=0.001, use_colnames=True)
frequent_itemsets

,support,itemsets
0,0.001173,(AROMA ROASTER OVEN)
1,0.001486,(BI-MART DISTILLED WATER 1 GAL.)
2,0.001642,(BI-MART DRAW-STRING TALL KITCHEN TRASH BAGS 6...
3,0.001069,(BOBS RED MILL UNBLEACHED WHITE FLOUR 5 LB)
4,0.011235,(BORE BUTTER NATURAL LUBE 1000 PLUS)
5,0.011288,(BUTLER CREEK BIKINI SCOPE COVER)
6,0.003024,(CARAVAN CANOPY V SERIES 2 PRO CANOPY)
7,0.528479,(CASCADE MEADOWS 1.1 CU FT DIGITAL MICROWAVE- ...
8,0.001356,(CASCADE OUTDOOR GEAR RAPID RIDER DOUBLE RIVER...
9,0.002815,(CASCADE OUTDOOR GEAR RAPID RIDER SINGLE RIVER...


In [21]:
rules = association_rules(frequent_itemsets, metric="lift", min_threshold=1)
rules.head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction


### Other Approach

In [20]:
# Recover transaction itemsets from orders DataFrame.
transactions = final_pro.groupby("order_number").product_name.unique()

In [21]:
transactions.head()

order_number
1000                       [COMMERCIAL CHEF .9 MICROWAVE]
1001                       [COMMERCIAL CHEF .9 MICROWAVE]
1002    [LOLA STICKY MOP REFILL, TERRO T300 ANT KILLER...
1003    [OUTDOOR PORTABLE GAS FIRE PIT, MEMBERSHIP: $5...
1004    [AUDUBON PARK WILD BIRD FOOD 10 LBS, PHILLIPS ...
Name: product_name, dtype: object

In [22]:
# Convert the pandas series to list of lists.
transactions = transactions.tolist()
# Print length of transactions.
print(len(transactions))

# Count number of unique item categories for each transaction.
counts = [len(transaction) for transaction in transactions]

# Print median number of items in a transaction.
print(np.median(counts))

# Print maximum number of items in a transaction.
print(np.max(counts))



38361
1.0
60


In [23]:
from mlxtend.preprocessing import TransactionEncoder

# Instantiate an encoder.
encoder = TransactionEncoder()

# Fit encoder to list of lists.
encoder.fit(transactions)

# Transform lists into one-hot encoded array.
onehot = encoder.transform(transactions)

# Convert array to pandas DataFrame.
onehot = pd.DataFrame(onehot, columns = encoder.columns_)
onehot.head()

,#10-24 INSERT NUT #880548,#10-24 X 2 ZINC THUMB SCREW #880947,#6 X 5/8 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882624,#8 X 3/4 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882625,1 AMP USB WALL CHARGER BLACK,1 INCH BRASS PLATED NON-REMOVABLE PIN HINGES 2 COUNT,1 PACK FLUIDMASTER 400CRP14 TOILET FILL VALVE AND FLAPPER REPAIR KIT,1-1/16 INCH THREADED FURNITURE LEVELING GLIDES 4 COUNT,1-1/2 INCH BROWN ROUND SELF-STICK FELT PADS 24 COUNT,1-1/8 INCH OATMEAL NAIL-ON FURNITURE GLIDES WITH FELT 8 COUNT,...,ZIPLOC TWIST N LOCK STORAGE CONTAINER 16 OZ 3 COUNT,ZONE PERFECT KETO BARS CHOCOLATE HAZELNUT 5 PK,ZOOM 001-004 U-TALE WORM PLUM,ZOOM 011045 FAT ALBERT GRUB WHITE PEARL,"ZOTZ FIZZ POWER CANDY, .7OZ STRIP CANDY (FLAVORS VARY)","ZOTZ FIZZ, POWER CANDY STRINGS, 0.7 OUNCE (FLAVORS VARY)",ZYRTEC ALLERGY RELIEF 24 CT,ZYRTEC ANTIHISTAMINE 10 MG LIQUID GELS 25 CT,membership_product,roadpower PREMIUM BATTERY TESTER
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [24]:
# Print support metric over all rows for each column.
print(onehot.mean(axis=0))

#10-24 INSERT NUT #880548                                               0.000026
#10-24 X 2 ZINC THUMB SCREW #880947                                     0.000052
#6 X 5/8 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882624    0.000026
#8 X 3/4 BLACK CHROMATE OVAL-HEAD PHILLIPS SHEET METAL SCREW #882625    0.000026
1 AMP USB WALL CHARGER BLACK                                            0.000026
                                                                          ...   
ZOTZ FIZZ, POWER CANDY STRINGS, 0.7 OUNCE (FLAVORS VARY)                0.000052
ZYRTEC ALLERGY RELIEF 24 CT                                             0.000026
ZYRTEC ANTIHISTAMINE 10 MG LIQUID GELS 25 CT                            0.000026
membership_product                                                      0.001225
roadpower PREMIUM BATTERY TESTER                                        0.000026
Length: 8494, dtype: float64


In [25]:
# Print distribution of item counts.
onehot.sum(axis=1).value_counts()

1     34077
2      1931
3       690
4       432
5       295
6       184
7       155
8       107
9        99
10       71
11       70
12       51
13       32
14       28
16       24
15       19
18       13
20       11
21       10
17       10
19       10
22        7
23        6
30        3
44        3
24        3
35        2
31        2
27        2
28        2
47        1
39        1
49        1
38        1
50        1
25        1
36        1
26        1
60        1
33        1
29        1
32        1
dtype: int64

In [26]:
#items_data['count'] = 1

In [27]:
#a = pd.DataFrame(items_data.groupby(['order_number'])['product_code'])

In [30]:
# Apply apriori algorithm to data with min support threshold of 0.01.
frequent_itemsets = apriori(onehot, min_support = 0.01)

# Print frequent itemsets.
frequent_itemsets

,support,itemsets
0,0.011235,(1292)
1,0.011288,(1468)
2,0.528479,(1655)
3,0.010479,(4235)
4,0.087798,(4832)
5,0.010010,(4852)
6,0.010714,(4873)
7,0.010427,(5267)
8,0.010245,(6789)
9,0.010505,(8056)


In [31]:
# Apply apriori algorithm to data with min support threshold of 0.001.
frequent_itemsets = apriori(onehot, min_support = 0.001, use_colnames = True)

# Print frequent itemsets.
frequent_itemsets

,support,itemsets
0,0.001173,(AROMA ROASTER OVEN)
1,0.001486,(BI-MART DISTILLED WATER 1 GAL.)
2,0.001642,(BI-MART DRAW-STRING TALL KITCHEN TRASH BAGS 6...
3,0.001069,(BOBS RED MILL UNBLEACHED WHITE FLOUR 5 LB)
4,0.011235,(BORE BUTTER NATURAL LUBE 1000 PLUS)
5,0.011288,(BUTLER CREEK BIKINI SCOPE COVER)
6,0.003024,(CARAVAN CANOPY V SERIES 2 PRO CANOPY)
7,0.528479,(CASCADE MEADOWS 1.1 CU FT DIGITAL MICROWAVE- ...
8,0.001356,(CASCADE OUTDOOR GEAR RAPID RIDER DOUBLE RIVER...
9,0.002815,(CASCADE OUTDOOR GEAR RAPID RIDER SINGLE RIVER...


In [28]:
# Apply apriori algorithm to data with min support threshold of 0.00005.
frequent_itemsets = apriori(onehot, min_support = 0.0001,verbose=1,low_memory=True, use_colnames = True)

# Print frequent itemsets.
frequent_itemsets

Processing 10 combinations | Sampling itemset size 9


,support,itemsets
0,0.000339,(13 LBS PURINA KIT & KABOODLE DRY CAT FOOD)
1,0.000261,(20/10 PRODUCTS WINDSHIELD WASH DEBUG 1 GALLON)
2,0.000156,(20/10 TT-W/W25-1X6 ALL SEASON WINDSHIELD WASH)
3,0.000104,"(3 MUSKETEERS, CANDY BARS, 1.92 OZ)"
4,0.000104,(3 PAIR MENS CUSHION QUARTER SOCKS. WHITE WITH...
...,...,...
4209,0.000104,"(MOUNTAIN HOUSE PAD THAI WITH CHICKEN, MOUNTAI..."
4210,0.000104,"(MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE, MOUNT..."
4211,0.000104,"(MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE, MOUNT..."
4212,0.000104,"(MOUNTAIN HOUSE MEXICAN ADOBO RICE & CHICKEN, ..."


In [31]:
# Recover association rules using confidence threshold of 0.01.
rules = association_rules(frequent_itemsets, metric = 'confidence', min_threshold = 0.01)

# Print rules.
rules = rules.sort_values(by='confidence',ascending=False)

rules.head(5)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
43651,"(MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE, MOUNT...","(MOUNTAIN HOUSE BREAKFAST SKILLET, MOUNTAIN HO...",0.000130,0.000261,0.000130,1.0,3836.100000,0.000130,inf
17019,"(MOUNTAIN HOUSE CHICKEN TERIYAKI, MOUNTAIN HOU...","(MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON, MOU...",0.000104,0.000261,0.000104,1.0,3836.100000,0.000104,inf
17016,"(MOUNTAIN HOUSE CHICKEN TERIYAKI, MOUNTAIN HOU...","(MOUNTAIN HOUSE BEEF STEW, MOUNTAIN HOUSE BEEF...",0.000104,0.000313,0.000104,1.0,3196.750000,0.000104,inf
17010,"(MOUNTAIN HOUSE BEEF STEW, MOUNTAIN HOUSE CHIC...",(MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON),0.000104,0.000365,0.000104,1.0,2740.071429,0.000104,inf
17009,"(MOUNTAIN HOUSE CHICKEN TERIYAKI, MOUNTAIN HOU...",(MOUNTAIN HOUSE BEEF STROGANOFF),0.000104,0.000704,0.000104,1.0,1420.777778,0.000104,inf


In [32]:
rules = rules.rename(columns={'antecedents':'antecedents_product_name','consequents':'consequents_product_name'})

In [33]:
rules = rules.head(1000)

In [34]:
from tqdm import tqdm
tqdm.pandas()

In [35]:
prod_code_name = items_data[['product_name','product_code']].drop_duplicates()
prod_code_name = dict(zip(items_data['product_name'],items_data['product_code']))


def product_code(x,prod_code_name,prod_url):
    prod=list(x)
    res=[]
    url = []
    if len(prod)>0:
        for p in prod:
            res.append(prod_code_name[p])
            if p in list(prod_url.keys()):
                if prod_url[p] is not None:
                    url.append(prod_url[p])
                else:
                    url.append('None')
            else:
                url.append('None')
    res = ','.join(res)
    prod = ','.join(prod)
    url = ','.join(url)
    return res,prod,url
rules['antecedents_product_id'],rules['antecedents_product_name'],rules['antecedents_image_url'] = zip(*rules['antecedents_product_name'].progress_apply(lambda x : product_code(x,prod_code_name,prod_url)))
rules['consequent_product_id'],rules['consequents_product_name'],rules['consequents_image_url'] = zip(*rules['consequents_product_name'].apply(lambda x : product_code(x,prod_code_name,prod_url)))
rules['tenant_id'] = 'TNB00084'
rules['store_id'] = 'STOM000000123'
rules['rules']=rules.index+1

100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:06<00:00, 159.60it/s]


In [36]:
rules.columns

Index(['antecedents_product_name', 'consequents_product_name',
       'antecedent support', 'consequent support', 'support', 'confidence',
       'lift', 'leverage', 'conviction', 'antecedents_product_id',
       'antecedents_image_url', 'consequent_product_id',
       'consequents_image_url', 'tenant_id', 'store_id', 'rules'],
      dtype='object')

In [37]:
req_col = ['rules','antecedents_product_id','consequent_product_id',
               'antecedents_product_name','consequents_product_name',
              'antecedent support', 'consequent support', 'support', 'confidence',
       'lift', 'leverage', 'conviction','antecedents_image_url','consequents_image_url','tenant_id', 'store_id']

rules = rules[req_col]

In [38]:
rules = rules.rename(columns={'antecedent support':'antecedent_support','consequent support':'consequent_support'})
rules.to_csv('rules_new2_1000.csv',index=False)

In [39]:
rules.head(3)

,rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id
43651,43652,"847234,847239,847235,872125,847230","847237,847215","MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE,MOUNTAI...","MOUNTAIN HOUSE BREAKFAST SKILLET,MOUNTAIN HOUS...",0.000130,0.000261,0.000130,1.0,3836.10,0.000130,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17019,17020,"847222,847237,847235","847239,847215","MOUNTAIN HOUSE CHICKEN TERIYAKI,MOUNTAIN HOUSE...","MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON,MOUNT...",0.000104,0.000261,0.000104,1.0,3836.10,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17016,17017,"847222,847239,847237","847215,847235","MOUNTAIN HOUSE CHICKEN TERIYAKI,MOUNTAIN HOUSE...","MOUNTAIN HOUSE BEEF STEW,MOUNTAIN HOUSE BEEF S...",0.000104,0.000313,0.000104,1.0,3196.75,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123


In [40]:
rules.shape

(1000, 16)

In [52]:
query_col = """(rules INT,antecedents_product_id CHAR(1000),consequent_product_id CHAR(1000),
                antecedents_product_name VARCHAR(5000),consequents_product_name VARCHAR(5000),
                antecedent_support FLOAT,consequent_support FLOAT,support FLOAT,confidence FLOAT,
                lift FLOAT,leverage FLOAT,conviction VARCHAR(1000),
                antecedents_image_url VARCHAR(5000),consequents_image_url VARCHAR(5000),tenant_id VARCHAR(1000),store_id VARCHAR(1000))"""

# query = """
# CREATE TABLE cdp.often_bundled_rules (rules INT, antecedents CHAR(1000), consequents CHAR(1000) ,
#                     antecedent_support FLOAT, consequent_support FLOAT, support FLOAT, confidence FLOAT,
#                     lift FLOAT, leverage FLOAT, conviction FLOAT, tenant_id VARCHAR(1000), store_id VARCHAR(1000));

# """
query = """
CREATE TABLE cdp.often_bundled_rules  
"""+query_col
cur.execute("ROLLBACK")
con.commit()
cur.execute(query)
con.commit()

In [51]:
# cur.execute("ROLLBACK")
# con.commit()
# drop_query = """ DROP TABLE cdp.often_bundled_rules """
# cur.execute(drop_query)
# con.commit()

In [ ]:
# Here you want to change your database, username & password according to your own values
param_dic = {
    "host"      : "mantix-cluster.cmpdmnkfrh2m.us-east-2.redshift.amazonaws.com",
    "database"  : "dev",
    "user"      : "mantix",
    "password"  : "Mantix!2"
}

#import psycopg2
#con=psycopg2.connect(dbname= 'dev', host='mantix-cluster.cmpdmnkfrh2m.us-east-2.redshift.amazonaws.com', 
#port= '5439', user= 'mantix', password= 'Mantix!2')

from sqlalchemy import create_engine
connect = "postgresql+psycopg2://%s:%s@%s:5439/%s" % (
    param_dic['user'],
    param_dic['password'],
    param_dic['host'],
    param_dic['database']
)
engine = create_engine(connect)

rules.to_sql(
        name='often_bundled_rules',schema='cdp',
    
        con=engine, 
        index=False,
       if_exists='replace'
    )
print("to_sql() done (sqlalchemy)")

In [41]:
#con.commit()
pd.read_sql('select * from cdp.often_bundled_rules ',con)

,rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id


In [49]:
rules.head()

,rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id
43651,43652,"847234,847239,847235,872125,847230","847237,847215","MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE,MOUNTAI...","MOUNTAIN HOUSE BREAKFAST SKILLET,MOUNTAIN HOUS...",0.000130,0.000261,0.000130,1.0,3836.100000,0.000130,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17019,17020,"847222,847237,847235","847239,847215","MOUNTAIN HOUSE CHICKEN TERIYAKI,MOUNTAIN HOUSE...","MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON,MOUNT...",0.000104,0.000261,0.000104,1.0,3836.100000,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17016,17017,"847222,847239,847237","847215,847235","MOUNTAIN HOUSE CHICKEN TERIYAKI,MOUNTAIN HOUSE...","MOUNTAIN HOUSE BEEF STEW,MOUNTAIN HOUSE BEEF S...",0.000104,0.000313,0.000104,1.0,3196.750000,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17010,17011,"847215,847222,847237,847235",847239,"MOUNTAIN HOUSE BEEF STEW,MOUNTAIN HOUSE CHICKE...",MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON,0.000104,0.000365,0.000104,1.0,2740.071429,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123
17009,17010,"847222,847239,847237,847215",847235,"MOUNTAIN HOUSE CHICKEN TERIYAKI,MOUNTAIN HOUSE...",MOUNTAIN HOUSE BEEF STROGANOFF,0.000104,0.000704,0.000104,1.0,1420.777778,0.000104,inf,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,https://cdn-tp3.mozu.com/29920-49677/cms/49677...,TNB00084,STOM000000123


In [ ]:
def single_insert(conn, insert_req):
    """ Execute a single INSERT request """
    cursor = conn.cursor()
    try:
        cursor.execute(insert_req)
        conn.commit()
    except (Exception, psycopg2.DatabaseError) as error:
        print("Error: %s" % error)
        conn.rollback()
        cursor.close()
        return 1
    cursor.close()

# Inserting each row
for i in rules.index:
    query = """
    INSERT into cdp.often_bundled_rules(rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id) 
        values('%s',%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);
    """ % (rules.loc[i]['rules'], 
           "'"+rules.loc[i]['antecedents_product_id']+"'", 
           "'"+rules.loc[i]['consequent_product_id']+"'",
           "'"+rules.loc[i]['antecedents_product_name']+"'",
           "'"+rules.loc[i]['consequents_product_name']+"'",
           rules.loc[i]['antecedent_support'],
           rules.loc[i]['consequent_support'],
           rules.loc[i]['support'],
           rules.loc[i]['confidence'],
           rules.loc[i]['lift'],
           rules.loc[i]['leverage'],
            "'"+str(rules.loc[i]['conviction'])+"'",
           "'"+rules.loc[i]['antecedents_image_url']+"'",
           "'"+rules.loc[i]['consequents_image_url']+"'",
           "'"+str(rules.loc[i]['tenant_id'])+"'",
           "'"+str(rules.loc[i]['store_id'])+"'")
#     query = """INSERT into test(rules, antecedents, consequents,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,teneat_id,store_id) 
#     values (1,2,3,4,5,6,7,8,9,10,11,12)
#     """
    print(query)
    #print(rules.loc[i]['antecedents'])
    single_insert(con, query)
# Close the connection
#con.close()
con.commit()


    INSERT into cdp.often_bundled_rules(rules,antecedents_product_id,consequent_product_id,antecedents_product_name,consequents_product_name,antecedent_support,consequent_support,support,confidence,lift,leverage,conviction,antecedents_image_url,consequents_image_url,tenant_id,store_id) 
        values('43652','847234,847239,847235,872125,847230','847237,847215','MOUNTAIN HOUSE LASAGNA WITH MEAT SAUCE,MOUNTAIN HOUSE SCRAMBLED EGGS WITH BACON,MOUNTAIN HOUSE BEEF STROGANOFF,MOUNTAIN HOUSE FETTUCCINE ALFREDO WITH CHICKEN,MOUNTAIN HOUSE CHICKEN FAJITA BOWL','MOUNTAIN HOUSE BREAKFAST SKILLET,MOUNTAIN HOUSE BEEF STEW',0.00013034071061755428,0.00026068142123510855,0.00013034071061755428,1.0,3836.1000000000004,0.0001303067332158657,'inf','https://cdn-tp3.mozu.com/29920-49677/cms/49677/files/d700b4dc-3a42-4f5d-b1bb-d22f8adcf3dc,https://cdn-tp3.mozu.com/29920-49677/cms/49677/files/b617cb2c-733f-42c6-83a5-905144c2b5d4,https://cdn-tp3.mozu.com/29920-49677/cms/49677/files/8b4207dd-fa28-4564-b114-2c